<a href="https://colab.research.google.com/github/momohirata/study_tensorflow/blob/master/study_mnist_chapter3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
#-*- coding:utf-8 -*-
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
from google.colab import drive

## MNISTデータを格納したオブジェクトを呼び出す
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)



#------------順伝播----------------
# 入力データを定義
# [ミニバッチサイズ(まだわからないのでNone), 1枚の画像の画素数(画素数)]の行列
x = tf.placeholder(tf.float32, [None,784]) 


# 入力画像をログに出力
# 入力を一旦整形してtf.summary.image()というオペレーションの入力にする
# [ミニバッチサイズ, 縦, 横, チャンネル数]
# チャンネル数: グレースケール1, RGB 3, RGBA 4
img = tf.reshape(x,[-1,28,28,1])
tf.summary.image("input_data", img, 10)


### 入力層から中間層
# name_scope: 各層や処理単位ごとに整理してまとめる機能
with tf.name_scope("hidden"):
    w_1 = tf.Variable(tf.truncated_normal([784,64],stddev=0.1),name="w1")
    b_1 = tf.Variable(tf.zeros([64]),name="b1")
    h_1 = tf.nn.relu(tf.matmul(x, w_1) + b_1)

    # 中間層の重みの分布をログ出力
    tf.summary.histogram('w_1', w_1)


### 中間層から出力層
with tf.name_scope("output"):
    w_2 = tf.Variable(tf.truncated_normal([64, 10], stddev=0.1), name="w2")
    b_2 = tf.Variable(tf.zeros([10]), name = "b2")
    out = tf.nn.softmax(tf.matmul(h_1, w_2) + b_2)


# 正解データ[None(ミニバッチサイズ), 出力層のユニット数]
y = tf.placeholder(tf.float32, [None, 10])


### 誤差関数
# 出力結果との差を求める → それを2乗して二乗誤差を求める → 全二乗誤差を平均
with tf.name_scope("loss"):
    loss = tf.reduce_mean(tf.square(y - out))
    
    # 誤差をログ出力
    tf.summary.scalar("loss", loss)

    
### 訓練
# 確率的勾配降下法(学習率:0.5)
with tf.name_scope("train"):
    train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)



### 評価
# 正解データ、出力データから最大値は何番目かを取得(一番確率の列(0~9)を求める)
# [バッチサイズ, 1]の行列を作成 → 出力データが正解かどうか判定 → 正解率を求める
with tf.name_scope("accuracy"):
    correct = tf.equal(tf.argmax(out, 1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

    # 精度をログ出力
    tf.summary.scalar("accuracy", accuracy)

    

# 実行

# 変数の初期化
init = tf.global_variables_initializer()

# すべてのログをmerge
summary_op = tf.summary.merge_all()


with tf.Session() as sess:
    
    summary_writer = tf.summary.FileWriter('logs', sess.graph)
    
    #初期化を実行
    sess.run(init)
    
    # テスト用データをロード
    test_images = mnist.test.images
    test_labels = mnist.test.labels
  
    for step in range(1000):
        train_images, train_labels = mnist.train.next_batch(50) #ミニバッチサイズ50
        # ミニバッチ取得 → placeholderへ渡す → train_step実行
        sess.run(train_step, feed_dict={x:train_images ,y:train_labels})
    
        # 10stepごとに精度を確認
        if step % 10 == 0:
            # ログを取る処理を実行
            summary_str = sess.run(summary_op, feed_dict={x:train_images ,y:train_labels})
            # ログ情報のプロトコルバッファを書き込む
            #summary_writer.add_summary(summary_str, step)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


InvalidArgumentError: ignored

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
